#Scripted Primo Search Assessment Tool

Description: < Describe the purpose of the SPSAT>

## Load required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import glob
import os
import codecs
import json
import time
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus

##Define Some Variables

In [2]:
## URLs ##

# prod_url = <base url>
prod_url_base = 'http://bu-primo.hosted.exlibrisgroup.com:1701/PrimoWebServices/xservice/search/brief'
# stage_url = <base url>
stage_url_base = 'http://bu-primostage.hosted.exlibrisgroup.com:1701/PrimoWebServices/xservice/search/brief'

## query params ##
query_Params1 = '?institution=BOSU&query=any,contains,'
query_Params2 = '&indx=1&bulkSize=10&loc=local,scope,BOSU&loc=adaptor,primo_central_multiple_fe&onCampus=true&json=true'
stage = True
#production = True
#num_results = 10
#scope = 

## file locations ## 

# move to working directory, this is the path where we want files to reside
os.chdir('/Volumes/jwa_drive1/git/spst')

# Input file (contains search strings)
f = '/Volumes/jwa_drive1/git/spst/search_strings.csv'

##Define functions

In [3]:
# build_url 
    # takes base_url and search_string, returns url properly formated for searching
def build_url(base_url,search_string):
    url = base_url+query_Params1 + search_string.replace(' ','+') + query_Params2
    return(url)
    
# parse_response 
    # takes response (as json) from search, returns response_dict (dict of our responses)
def parse_response(search,response):
    response_dict = {}
    # parse the response to return the data desired elements
    response = json.loads(response.decode('utf8'))
    # need to extract the desired elements and add them to a dict
    response = response['SEGMENTS']
    response = response['JAGROOT']
    response = response['RESULT']
    response_dict['facet'] = response['FACETLIST']
    response_dict['docset'] = response['DOCSET']
    docs = response_dict['docset']['DOC']
    ## define a dataframe to return the results
    ret_columns = ['Search','ResultNumber','TotalHits','FirstHit','LastHit','Rank','SearchEngine','Bib']
    df_return = pd.DataFrame(columns = ret_columns) 
    ## iterate through the results (normally 10) to populate a dictionary that will be added to the dataframe
    for doc in docs:
        doc_dict = {}
        doc_dict['Search'] = search
        doc_dict['ResultNumber'] = doc['@NO']
        doc_dict['TotalHits'] = response['DOCSET']['@TOTALHITS']
        doc_dict['FirstHit'] = response['DOCSET']['@FIRSTHIT']
        doc_dict['LastHit'] = response['DOCSET']['@LASTHIT']
        doc_dict['Rank'] = doc['@RANK']
        doc_dict['SearchEngine'] = doc['@SEARCH_ENGINE']
        doc_dict['Bib'] = doc['PrimoNMBib']['record']
        df_return = df_return.append(doc_dict, ignore_index=True)
    return(df_return)

def get_value(response, key):
    # find or navigate response until the key is found
    # then return the value associated with the key
    value = 'hello world'
    
    return(value)
    

In [4]:
# open input file, read each line as a separate string, and remove the new line character
search_strings = open(f).read().splitlines()

In [5]:
# create a DataFrame. Expand the columns to match the desired elements
# to be returned by the function 'parse_response'

# Create a dataframe with
# - column for search string 
# - column for response (returned from parse_response fuction)
columns = ['Search','ResultNumber','TotalHits','FirstHit','LastHit','Rank','SearchEngine','Bib']
df = pd.DataFrame(columns = columns)


# Loop through search_strings, execute search, ... [to be done?]
for search_string in search_strings:
    time.sleep(1)
    
    
    # can we use build_url here? 
    #search_url = stage_url_base+query_Params1 + search_url.replace(' ','+') + query_Params2
    search_url = build_url(prod_url_base,search_string)
    request = Request(search_url)
    # maybe we could do ... 
    # request = Request(build_url(stage_url_base,search_url))
    
    
    try:
        response_body = urlopen(request).read() # reads request into response_body var
        d = parse_response(search_string,response_body)
        df = df.append(d, ignore_index=True)
    #prints errors 
    except Exception as e:
        print(e)

In [7]:
# verify dataframe
df.head(20)

,Search,ResultNumber,TotalHits,FirstHit,LastHit,Rank,SearchEngine,Bib
0,murder,1,2964097,1,10,0.07,primo_central_multiple_fe,"{'display': {'language': 'eng', 'creator': 'Er..."
1,murder,2,2964097,1,10,0.040000003,primo_central_multiple_fe,{'display': {'identifier': '<b>ISSN: </b>0923-...
2,murder,3,2964097,1,10,0.035001222,primo_central_multiple_fe,{'display': {'identifier': '<b>ISSN: </b>08978...
3,murder,4,2964097,1,10,0.035,primo_central_multiple_fe,{'display': {'identifier': '<b>ISSN: </b>00142...
4,murder,5,2964097,1,10,0.025,primo_central_multiple_fe,{'display': {'identifier': '<b>ISSN:</b> 00195...
5,murder,6,2964097,1,10,0.025,primo_central_multiple_fe,{'display': {'identifier': '<b>ISSN:</b> 00195...
6,murder,7,2964097,1,10,0.025,primo_central_multiple_fe,{'display': {'identifier': '<b>ISSN:</b> 00363...
7,murder,8,2964097,1,10,0.020000534,primo_central_multiple_fe,{'display': {'identifier': '<b>ISSN: </b>00287...
8,murder,9,2964097,1,10,0.020000001,primo_central_multiple_fe,{'display': {'identifier': '<b>ISSN:</b> 00035...
9,murder,10,2964097,1,10,0.01250014,primo_central_multiple_fe,"{'display': {'language': 'eng', 'type': 'artic..."


In [7]:
for index,row in df.iterrows():
    print('SearchString: ',row['Search'])
    print('ResultNumber: ',row['ResultNumber'])
    print('TotalHits: ',row['TotalHits'])
    print('Rank: ',row['Rank'])
    print('SearchEngine: ',row['SearchEngine'])  
    print('Bib: ',row['Bib'])
    print()

SearchString:  murder
ResultNumber:  1
TotalHits:  2964097
Rank:  0.07
SearchEngine:  primo_central_multiple_fe
Bib:  {'search': {'creationdate': '2014', 'subject': 'Torture', 'addtitle': 'University Wire', 'description': "\xa0 [...]you can see why the discovery of her preferred murdering style is especially alarming to me, when she actually can trap me in a tiny room until I die. Since I have strong self-preserving instincts I will of course do everything I can to stay in 3's good graces, but I'm also composing my last will and testament and choosing a casket, just to be safe.", 'title': 'Murder', 'rsrctype': 'newspaper_article', 'searchscope': ['1006080', '1006454', '1006769', '1006907', '1007161', '1007945', '1007989', '1007990', '1007991', '1007992', '1007993', '1007994', '1007995', '1008503', '10000039', '10000062', '10000118', '10000164', '10000184', '10000210', '10000213', 'proquest'], 'sourceid': 'proquest', 'startdate': '20141208', 'recordid': 'proquest1645937272', 'enddate': 